# Sezione 4: regolarizzazione e modelli regolarizzati

## lezioni 27 - 30

Introduciamo due nuovi concetti: il Bias e la varianza
#### Bias: 
Non è da intendersi come l'intercetta del nostro modello, ma è una metrica che misura quanto distanti sono le nostre previsioni da quelle corrette, se dovessimo costruire piu volte il modello utilizzando siversi dataset. In pratica misura l'errore sistematico del modello che non è dato dalla casualita dei dati che gli vengono forniti.

#### Varianza:
E' una metrica che misura la differenza delle predizioni se addestriamo piu volte il modello du diversi dataset.
In pratica ci dice quanto il modello è sensibile alla casualita de dati del dataset di addestramento, il trainingset.

Bias e varianza sono altamente correlati: infatti se aumenta uno diminuisce l'altro.
Il nostro lavoro è di riuscire a trovare la giusta combinazione per ottenere bias e varianza che siano entrami i piu piccoli possibile

Nel video della lezione si ha un esempio di underfitting e di overfitting:
    
si ha underfitting se la varianza è bassa ma il bias è alto, vuol dire che il modello è troppo semplice e generico.

Si ha overfitting se la varianza è elevata e il bias basso, vuol dire che il modello è troppo complesso, e funziona bene solo per i dati con cui è satto allenato, il trainingset, mentre se lo si fa lavorare con altri dataset lavorera male.

Come riconoscere l'overfitting?
E' molto semplice, infatti dopo aver addestrato il modello basta confrontare l'errore sul set di addestramento con quello sul set di test. Se quest ultimo è molto piu grande, allora molto probabilmente il nostro modello soffre di overfitting.

Come possiamo contrastare l'overfitting?
Bisogna semplificare il modello, rimuovendo alcune proprieta. Nel nostro caso potremmo provare a ridurre il grado del polinomio.

Questa soluzione è semplice, ma non funziona sempre perche si rischia di perdere informazioni importanti insieme alle proprieta che decidiamo di escludere.

La seconda soluzione, che funziona meglio, consiste nel regolarizzare il nostro modello, penalizzando i pesi delle proprieta, applicando alcune tecniche che ora vediamo.

## Riconoscere l'overfitting


l'overfitting è un problema che si manifesta nel momento in cui un modello eccessivamente complesso non riesce a generalizzare su dati non visti in addestramento. 
Ora vediamo come riconoscerlo.

In [1]:
# iniziamo caricando le librerie necessarie:

import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


In [2]:
# carichiamo il dataset all'interno di un dataframe:

boston = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data", sep='\s+', 
                     names=["CRIM", "ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PRATIO","B","LSTAT","MEDV"])
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
#e creiamo gli array numpy per addestrament e test:

X = boston.drop('MEDV',axis=1).values
Y = boston['MEDV'].values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=0)

proviamo a creare un problema di overfitting:
per farlo creiamo nuove proprieta polinomiali per ogni proprieta del dataset:

In [4]:
poly_feats = PolynomialFeatures(degree=2)
X_train_poly = poly_feats.fit_transform(X_train)
X_test_poly = poly_feats.transform(X_test)

In [5]:
# vedaimo la dimensione del nostro test di addestramento:
X_train_poly.shape

(354, 105)

354 esempi e 105 proprieta: è molto complesso.

In [6]:
# Prima di eseguire la regressione, standardizziamo il dataset:

ss = StandardScaler()
X_train_poly = ss.fit_transform(X_train_poly)
X_test_poly = ss.transform(X_test_poly)

In [7]:
# ora eseguiamo la nostra regressione lineare.

ll = LinearRegression()
ll.fit(X_train_poly, Y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
# adesso eseguiamo la predizione, ma la facciamo sul trainset piuttosto che sul testset:

Y_pred_train = ll.predict(X_train_poly)

In [9]:
# adesso calcoliamo l'errore quadratico medio
#e il coefficiente di indeterminazione sulla predizione fatta sul set di addestramento:

mse_train = mean_squared_error(Y_train, Y_pred_train)
R2_train = r2_score(Y_train, Y_pred_train)

In [10]:
print("Train set: MSE= " + str(mse_train) + "  R2= " + str(R2_train))

Train set: MSE= 4.116985074931599  R2= 0.9514303225914083


L'errore è molto basso e il punteggio è molto alto.
Questo ci potrebbe far pensare di aver trovato un modello ottimo, ma in realta potremmo avere overfitting. 
Per verificarlo dobbiamo fare la predizione sul testset, calcolare errore e punteggio, e confrontiamo con quelli relativi al trainset.

In [11]:
Y_pred_test = ll.predict(X_test_poly)

mse_test = mean_squared_error(Y_test, Y_pred_test)
R2_test = r2_score(Y_test, Y_pred_test)

print("Test set: MSE= " + str(mse_test) + "  R2= " + str(R2_test))


Test set: MSE= 29.550444547162094  R2= 0.6451057885504259


Sia MSE che R2 sono decisamente peggiori rispetto a quelli del test di addestramento. QUindi ci troviamo sicuramente in Overfitting

##  Regolarizzazione per risolvere il problema dell'overfitting

La regolarizzazione penalizza i pesi più grandi riducendo la complessita del modello, quindi riducendo la varianza e aumentando il bias, il che fa apparire il modello piu regolare e meno soggetto a fluttuazioni.
Le principali tecniche di regolarizzazione sono 2: Regolarizzazione L2 e regolarizzazione L1.
Entrambe funzionano aggiungendo una penalita ai pesi molto grandi alla funzione di costo durante l'addestramento.

Per usare la regolarizzazione è necessario che tutti i dati siano sulla stessa scala. Sarà quindi necessario standardizzare o normalizzare il dataset.

#### Regolarizzazione L2:
Detta anche weight decay, questo termine è dato dalla somma dei quadrati dei pesi. Questo nuovo termine costringera il grdient descend a cercare il minio di questa nuova funzione di costo per valori dei pesi piu piccoli.
Lambda è un altro iperparametro, e serve a controlare il peso che la regolarizzazione avra sul modello

#### Regolarizzazione L1:
 Il termine da aggiungere alla funzione di costo durante l'addeestramento è la somma deò valore assoluto dei pesi.
 Questo nuovo termine porta il peso delle proprieta meno importanti a zero. 
 Quindi dato che una proprieta con peso 0 viene totalmente esclusa dal modello, la regolarizzazione L1 ci permette di fare una selezione delle proprieta.
 
 
 
 
 
 Nella pratica la L2 fornisce quasi sempre risultati migliori della L1, ma una buona tecnica è di usarle entrambe in coppia.
 
 
 Abbiamo detto che Lambda è un iperparametro, e quindi va ottimizzato per il nostro modello.
 Come scegliere il suo valore?
 
 Se lambda = 0 , il termine di regolarizzazione sara uguale a 0, il che equivale a fare una regressione lineare regolare, e quindi non si risolve l'overfitting.
 Se lambda è troppo grande, il peso della pregolarizzazione sara troppo alto e la maggiorparte dei pesi verranno ridotti a 0, o in prossimita dello 0, il che porterebbe il modello in condizioni di underfitting.
La soluzione corretta consiste nel cercare lambda in un range che va da 10^-4, ovvero 0,0001 e 10.

# Ridge, Lasso ed ElasticNet

proveremo a contrastare l'overfitting usando la regolarizzazione che, come abbiamo visto, è una tecnica che ci permette  di ridurre la complessita di un modello dimunuendo la magnitudine dei coefficienti.

Un modello di Machine Learning che applica la regolarizzazione L2 è la Ridge Regression.

#### Ridge Regression

In [12]:
#importiamo la classe ridge da SciKitLearn:

from sklearn.linear_model import Ridge

Usando la regolarizzazione dobbiamo gestire un nuovo iperparametro: il parametro di regolarizzazione, detto Lambda (Nel caso di SciKitLearn viene chiamato Alpha).
Andiamo a cercare il nostro modello regolarizzato su diversi valori di alpha:

In [13]:
#creiamo un array di possibili valori di alpha, fra 0,0001 e 10:

alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]

In [19]:
#ora creiamo diversi modelli, uno per ogni valori di alpha, all'interno di un ciclo for

for alpha in alphas:
    print("ALPHA=" +str(alpha))
    model = Ridge(alpha=alpha)
    model.fit(X_train_poly, Y_train)
    
    # Dopo aver costruito il modello andiamo a valutarlo sia sul set di addestramento che sul set di test.
    #Quindi andiamo a fare le predizioni per entrambi i set:

    Y_pred_train = model.predict(X_train_poly)
    Y_pred_test = model.predict(X_test_poly)
    
    #Andiamo a calcolare l'errore quadratico medio e il coeff. di indeterminazione per entrambi i set:
    
    mse_train = mean_squared_error(Y_train, Y_pred_train)
    mse_test = mean_squared_error(Y_test, Y_pred_test)
    R2_train = r2_score(Y_train, Y_pred_train)
    R2_test = r2_score(Y_test, Y_pred_test)
    
    # e li stampiamo per poterli confrontare:
    
    print("Train set: MSE=" +str(mse_train) + "  R2="+ str(R2_train))
    print("Test set: MSE= " + str(mse_test) + " R2=" + str(R2_test))
    

ALPHA=0.0001
Train set: MSE=4.0992634048797  R2=0.9516393920395351
Test set: MSE= 28.91761846311459 R2=0.6527058878898564
ALPHA=0.001
Train set: MSE=4.1135025099452385  R2=0.9514714077677846
Test set: MSE= 28.420009267359223 R2=0.6586820627272876
ALPHA=0.01
Train set: MSE=4.208206127239228  R2=0.950354152286485
Test set: MSE= 26.8132950181357 R2=0.6779783405072352
ALPHA=0.1
Train set: MSE=4.747028830954056  R2=0.9439974508597039
Test set: MSE= 23.631755117362193 R2=0.7161879211610329
ALPHA=1
Train set: MSE=5.875947305341951  R2=0.9306791596529934
Test set: MSE= 17.634584627529463 R2=0.7882125937009337
ALPHA=10
Train set: MSE=8.81275552173785  R2=0.8960324885854234
Test set: MSE= 17.15971577477408 R2=0.79391566211913


Per Alpha (Lambda) fino a 1, l'algoritmo non riesce a risolvere l'overfitting, ma per alpha = 1 e 10 si raggiunge un R2 poco inferiore a 0.8

# Lasso Regression

Proviamo ora ad utilizzare il Lasso, che è un modello di regressine lineare che applica la regolarizzazione L1, e quindi tende  portare a zero il peso di proprieta poco importanti.

In [22]:
# importiamo la classe Lasso da SciKitLearn:

from sklearn.linear_model import Lasso

In [23]:
# facciamo la stessa cosa di prima, semplicemente cambiando il tip di modello da Ridge a Lasso:

for alpha in alphas:
    print("ALPHA=" +str(alpha))
    model = Lasso(alpha=alpha)
    model.fit(X_train_poly, Y_train)
    
    # Dopo aver costruito il modello andiamo a valutarlo sia sul set di addestramento che sul set di test.
    #Quindi andiamo a fare le predizioni per entrambi i set:

    Y_pred_train = model.predict(X_train_poly)
    Y_pred_test = model.predict(X_test_poly)
    
    #Andiamo a calcolare l'errore quadratico medio e il coeff. di indeterminazione per entrambi i set:
    
    mse_train = mean_squared_error(Y_train, Y_pred_train)
    mse_test = mean_squared_error(Y_test, Y_pred_test)
    R2_train = r2_score(Y_train, Y_pred_train)
    R2_test = r2_score(Y_test, Y_pred_test)
    
    # e li stampiamo per poterli confrontare:
    
    print("Train set: MSE=" +str(mse_train) + "  R2="+ str(R2_train))
    print("Test set: MSE= " + str(mse_test) + " R2=" + str(R2_test))
    

ALPHA=0.0001
Train set: MSE=5.391123652697097  R2=0.9363988132296843
Test set: MSE= 29.701776720601217 R2=0.6432883230881412
ALPHA=0.001
Train set: MSE=5.407317548867128  R2=0.936207767525449
Test set: MSE= 28.788018557306383 R2=0.654262353691998
ALPHA=0.01
Train set: MSE=6.0638588169003125  R2=0.9284622943178908
Test set: MSE= 22.93324201265647 R2=0.7245769068863099
ALPHA=0.1
Train set: MSE=11.833211121207535  R2=0.8603989967405071
Test set: MSE= 19.296152342816377 R2=0.7682575380960781
ALPHA=1
Train set: MSE=21.590985067091978  R2=0.7452827346818105
Test set: MSE= 27.25804314512913 R2=0.6726370152499754
ALPHA=10
Train set: MSE=84.76451346994796  R2=0.0
Test set: MSE= 83.76673764512785 R2=-0.0060197319476869016


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 948.9527113282645, tolerance: 3.000663776836158
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 886.5733620155494, tolerance: 3.000663776836158
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 504.93622041518574, tolerance: 3.000663776836158
  positive)


Notiamo che il punteggio migliore si è ottenuto con Alpha = 0.1, infatti R2 = 0.72
mentre per valori maggiori di alpha, per esempio alpha = 10, l'effetto della regolarizzazione è troppo intenso, molti pesi vengono ridotti a 0 e ne risentono le performance del modello.

La Regolarizzazione L2 ha dato risultati migliori.
In generale conviene usare entrambi i tipi di regolarizzazione, L2 e L1, insieme.
Un modello che ci consente di farlo è ElasticNet:

#### ElasticNet:

Questo modello fornisce un altro parametro aggiuntivo, l1_ratio, che indica a quale dei due tipi di regolarizzazione dare più importanza. Con un valore di 0.5 i due metodi verranno utilizzati con lo stesso peso

In [24]:
#importiamo la classe da SciKitLearn:

from sklearn.linear_model import ElasticNet

In [26]:
# facciamo la stessa cosa di prima, semplicemente cambiando il tip di modello da Ridge/Lasso a ElasticNet:

for alpha in alphas:
    print("ALPHA=" +str(alpha))
    
    # dobbiamo specificare il parametro l1_ratio
    model = ElasticNet(alpha=alpha, l1_ratio = 0.5)
    model.fit(X_train_poly, Y_train)
    
    # Dopo aver costruito il modello andiamo a valutarlo sia sul set di addestramento che sul set di test.
    #Quindi andiamo a fare le predizioni per entrambi i set:

    Y_pred_train = model.predict(X_train_poly)
    Y_pred_test = model.predict(X_test_poly)
    
    #Andiamo a calcolare l'errore quadratico medio e il coeff. di indeterminazione per entrambi i set:
    
    mse_train = mean_squared_error(Y_train, Y_pred_train)
    mse_test = mean_squared_error(Y_test, Y_pred_test)
    R2_train = r2_score(Y_train, Y_pred_train)
    R2_test = r2_score(Y_test, Y_pred_test)
    
    # e li stampiamo per poterli confrontare:
    
    print("Train set: MSE=" +str(mse_train) + "  R2="+ str(R2_train))
    print("Test set: MSE= " + str(mse_test) + " R2=" + str(R2_test))
    

ALPHA=0.0001
Train set: MSE=5.391059281137908  R2=0.9363995726460551
Test set: MSE= 29.466017582883698 R2=0.6461197374561638
ALPHA=0.001
Train set: MSE=5.463124643400415  R2=0.9355493894819877
Test set: MSE= 26.23899793865858 R2=0.6848755196286369
ALPHA=0.01
Train set: MSE=6.6699478752202905  R2=0.921312025490655
Test set: MSE= 15.784424726986327 R2=0.8104325991533368
ALPHA=0.1
Train set: MSE=12.092531251957974  R2=0.8573396960952864
Test set: MSE= 20.123693597792254 R2=0.7583189532244383
ALPHA=1
Train set: MSE=21.178857007859765  R2=0.7501447700119411
Test set: MSE= 27.923580301576493 R2=0.6646440632674491
ALPHA=10
Train set: MSE=70.28359861834348  R2=0.1708369960353573
Test set: MSE= 69.68198552608109 R2=0.16313498207951238


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 966.4573759845673, tolerance: 3.000663776836158
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1013.5668798465641, tolerance: 3.000663776836158
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 993.001922022791, tolerance: 3.000663776836158
  positive)


Questo modello ha ottenuto il punteggio migliore per Alpha 0.1.
Il risultato è migliore rispetto ai casi precedenti: l'errore è minore e il punteggio (il coeff. di indeterminazione) è maggiore, e l'overfitting è stato notevolmente ridotto.
 
Quando si usa la regolarizzazione per eliminare l'overfitting, è necessario che tutti i dati siano sulla stessa scala, e quindi bisogna standardizzare, come abbiamo fatto in questo caso, oppure normalizzare.